In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import pprint
from pylab import savefig

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ponta Do Sol,PT,32.6667,-17.1000,66.70,87,100,7.92,overcast clouds
1,1,Bluff,NZ,-46.6000,168.3333,74.80,53,30,12.97,scattered clouds
2,2,Busselton,AU,-33.6500,115.3333,88.59,35,29,9.33,scattered clouds
3,3,Tiksi,RU,71.6872,128.8694,-12.30,100,100,6.15,overcast clouds
4,4,Lorengau,PG,-2.0226,147.2712,83.79,79,81,9.06,light rain


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ponta Do Sol,PT,32.6667,-17.1000,66.70,87,100,7.92,overcast clouds
1,1,Bluff,NZ,-46.6000,168.3333,74.80,53,30,12.97,scattered clouds
2,2,Busselton,AU,-33.6500,115.3333,88.59,35,29,9.33,scattered clouds
4,4,Lorengau,PG,-2.0226,147.2712,83.79,79,81,9.06,light rain
5,5,Tautira,PF,-17.7333,-149.1500,82.42,78,27,16.75,light rain
6,6,Cape Town,ZA,-33.9258,18.4232,68.41,78,0,3.00,clear sky
9,9,Mar Del Plata,AR,-38.0023,-57.5575,63.09,88,0,5.99,clear sky
10,10,Cidreira,BR,-30.1811,-50.2056,71.60,83,0,17.16,clear sky
12,12,Avarua,CK,-21.2078,-159.7750,82.45,74,20,11.50,moderate rain
14,14,Rikitea,PF,-23.1203,-134.9692,76.46,79,100,20.11,moderate rain


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                357
City                   357
Country                355
Lat                    357
Lng                    357
Max Temp               357
Humidity               357
Cloudiness             357
Wind Speed             357
Current Description    357
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.count()

City_ID                355
City                   355
Country                355
Lat                    355
Lng                    355
Max Temp               355
Humidity               355
Cloudiness             355
Wind Speed             355
Current Description    355
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ponta Do Sol,PT,66.70,overcast clouds,32.6667,-17.1000,
1,Bluff,NZ,74.80,scattered clouds,-46.6000,168.3333,
2,Busselton,AU,88.59,scattered clouds,-33.6500,115.3333,
4,Lorengau,PG,83.79,light rain,-2.0226,147.2712,
5,Tautira,PF,82.42,light rain,-17.7333,-149.1500,
6,Cape Town,ZA,68.41,clear sky,-33.9258,18.4232,
9,Mar Del Plata,AR,63.09,clear sky,-38.0023,-57.5575,
10,Cidreira,BR,71.60,clear sky,-30.1811,-50.2056,
12,Avarua,CK,82.45,moderate rain,-21.2078,-159.7750,
14,Rikitea,PF,76.46,moderate rain,-23.1203,-134.9692,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
     # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().count()

City                   355
Country                355
Max Temp               355
Current Description    355
Lat                    355
Lng                    355
Hotel Name             355
dtype: int64

In [15]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ponta Do Sol,PT,66.70,overcast clouds,32.6667,-17.1000,Hotel do Campo
1,Bluff,NZ,74.80,scattered clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
2,Busselton,AU,88.59,scattered clouds,-33.6500,115.3333,Observatory Guest House
4,Lorengau,PG,83.79,light rain,-2.0226,147.2712,Lorengau Harbourside Hotel
5,Tautira,PF,82.42,light rain,-17.7333,-149.1500,Pension Oaoa
6,Cape Town,ZA,68.41,clear sky,-33.9258,18.4232,Southern Sun Waterfront Cape Town
9,Mar Del Plata,AR,63.09,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
10,Cidreira,BR,71.60,clear sky,-30.1811,-50.2056,Hotel Castelo
12,Avarua,CK,82.45,moderate rain,-21.2078,-159.7750,Paradise Inn
14,Rikitea,PF,76.46,moderate rain,-23.1203,-134.9692,People ThankYou


In [19]:
# 8a. Create the output File (CSV)
output_data_file= "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 

description=hotel_df["Current Description"]
fig= gmaps.figure(center=(30,31), zoom_level=1.4)
heat_layer= gmaps.heatmap_layer(locations, dissipating= False,
                               max_intensity=300, point_radius=4)
marker_layer= gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
pprint.pprint(hotels)

{'html_attributions': [],
 'next_page_token': 'Aap_uEBD3_ZkER5MtrPymfpBf-afzNzGB__MjF76_juDG11GQvlND0Egh8Adoa-WI5AL8gewxiMzY5fV1Me1w46tUACuenwUXIt1i_7RGjTodj_dqMJmEyQRhXq3x0ejV2tGrpzGLgJhNlDsJgbitPFrVBViGgzujA4Ns4DObkaDYBy3JY-BYmCjJQqmdtw9auyMi2Gyc1erljnsWsvf8dgcxs5dFMLaq6HFHtpOsh-eiOuy1Flmhvd55PrHLSyMszo6W-I9P5-pq5EE8_jg8uDyA2lMMOKW9Sk5KavzRFifEqzphGU5QRJugBx6IHh69p0O6gbbrowGJLc_I4i9WgzNbUjBFErWU0zEroS5wyJZ6NnZttMVl2r4EG53c-fxkrzZVkAad12NHRpbTw2gVwe51lmNTAluL9hFoFvD0GKVBMfFJd2ZvVdkTlP6',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': -21.6408235, 'lng': -41.0521534},
                           'viewport': {'northeast': {'lat': -21.6395370197085,
                                                      'lng': -41.05054781970851},
                                        'southwest': {'lat': -21.6422349802915,
                                                      'lng': -41.0532457802915}}},
              'icon': 'https://maps.gstatic.com/mapf